# SmoothQuant on OPT-13B

### Guangxuan Xiao\*, Ji Lin\*, Mickael Seznec, Julien Demouth, Song Han

In this notebook, we use OPT-13B model to demonstrate SmoothQuant can use 8-bit for both weights and activations to achieve the same accuracy as FP16 models. Unlike previous method [[Dettmers *et al.*, 2022]](https://arxiv.org/abs/2208.07339), SmoothQuant enables fully INT8 GEMMs for linear layers and does not require high precision numbers to represent outliers. 

This notebook demonstrates SmoothQuant on OPT-13B in consideration of the user's resouce constraints. We have tested SmoothQuant on up to 176 billion parameter models (OPT-175B, BLOOM-176B, GLM-130B). You can also adjust the model name to validate SmoothQuant on other models. `../act_scales/` provides the activation channel scales for OPT and BLOOM models.

In order to run this notebook, you need to install the following packages:

- smoothquant
- PyTorch
- Transformers
- Accelerate

In [1]:
import os
os.chdir('/Users/cqhhxw/Documents/TinyML/efficient_ml_class_project/')

# import importlib
# # Force reimport of the module
# importlib.reload(importlib.import_module("investigate"))
# importlib.reload(importlib.import_module("smoothquant.smoothquant.fake_quant"))

from investigate import sweep, report_sweep

repo_dir = "smoothquant"
short_model_name = "opt-125m"
sweep(short_model_name=short_model_name, repo_dir=repo_dir, save_dir=".")
report_sweep(short_model_name=short_model_name, save_dir=".")


# n_bits = 8
# q_group_size = 0 # 0 means no grouping
# q_protect = False # False means no protection
# q_protection_scale = 0.0 # 0.0 means mixed-precision. >= 1.0 means actual scale up/down.
# q_protection_ratio = 0.01 # 0.01 means 1% of the weights are protected.
# q_smoothing_strength = 0.5

# investigation = Investigation(short_model_name=short_model_name, repo_dir=repo_dir, n_bits=n_bits, q_group_size=q_group_size, q_protect=q_protect, q_protection_scale=q_protection_scale, q_protection_ratio=q_protection_ratio, q_smoothing_strength=q_smoothing_strength)


ModuleNotFoundError: No module named 'torch'

In [ ]:
# AWQ Investigate
short_model_name = "opt-125m"
repo_dir = "llm_awq"
awq_zoo = "mit-han-lab/awq-model-zoo"
awq_pt_name = f"{short_model_name}-w4-g128.pt"
from huggingface_hub import hf_hub_download
awq_pt_filename = hf_hub_download(repo_id=awq_zoo, filename=awq_pt_name, repo_type="dataset")
awq_pt_filename

from llm_awq.awq.quantize.pre_quant import apply_awq
import torch
awq_results = torch.load(awq_pt_filename, map_location="cpu")



In [ ]:
from smoothquant.smoothquant.investigate import sweep, report_sweep, make_setups

setups = make_setups()
print(setups[0])
print(setups[1])


In [ ]:
base_res = investigation.evaluate_base_model(perp=True)
print(f"Base Result: {base_res}")

In [ ]:
quantized_res = investigation.evaluate_quantized_model(perp=True)
print(f"Quantized Result: {quantized_res}")

In [ ]:
smooth_quantized_res = investigation.evaluate_smooth_model(perp=True)
print(f"Smooth Quantized Result: {smooth_quantized_res}")

In [ ]:
import torch
import tqdm
from torch import nn
from transformers.models.opt.modeling_opt import (
    OPTAttention,
    OPTDecoderLayer,
    OPTForCausalLM,
)
from transformers import GPT2Tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from smoothquant.smoothquant.smooth import smooth_lm
from smoothquant.smoothquant.fake_quant import WQAQLinear, quantize_opt

In this notebook, we simulate the 8-bit dynamic per-tensor weight and activation quantization with FP16, i.e., fake quantization. We have implemented the real 8-bit quantization with INT8 CUTLASS GEMM kernels for both PyTorch and FasterTransformer. Please stay tuned for the release.

The following is an evaluator to see the performance of the model. We use a toy dataset (the first 1000 examples in the validation set of the Lambada dataset) to evaluate the model. You can replace it with your own dataset. The conclusion should be the same.

**In this demo, we have simplified the evaluation by using the first 1,000 samples from the LAMBADA dataset's validation set. We employ the "Last Token Prediction Accuracy" as our evaluation metric. This approximate evaluation is intended for demonstration purposes, providing simple but meaningful comparisons of relative performance between methods. For a more strict assessment, we recommend using the [lm-eval-harness](https://github.com/EleutherAI/lm-evaluation-harness) to obtain the "Last Word Prediction Accuracy" for the LAMBADA dataset, which is the reported metric in our paper.**

In [ ]:
class PerplexityEvaluator:
    def __init__(self, dataset, tokenizer, device, n_samples=40):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        self.dataset = tokenizer(
            "\n\n".join(dataset["text"]), return_tensors="pt"
        ).input_ids.to(device)

        self.n_samples = n_samples

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        testenc = self.dataset
        nsamples = self.n_samples
        model = model.eval()

        nlls = []
        for i in tqdm.tqdm(range(nsamples), desc="evaluating..."):
            batch = testenc[:, (i * 2048):((i + 1) * 2048)].to(model.device)
            with torch.no_grad():
                lm_logits = model(batch).logits
            shift_logits = lm_logits[:, :-1, :].contiguous().float()
            shift_labels = testenc[:, (i * 2048):((i + 1) * 2048)][:, 1:]
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            neg_log_likelihood = loss.float() * 2048
            nlls.append(neg_log_likelihood)

        return torch.exp(torch.stack(nlls).sum() / (nsamples * 2048))

class AccuracyEvaluator:
    def __init__(self, dataset, tokenizer, device):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device

        # tokenize the dataset
        def tokenize_function(examples):
            example = self.tokenizer(examples["text"])
            return example

        self.dataset = self.dataset.map(tokenize_function, batched=True)
        self.dataset.set_format(type="torch", columns=["input_ids"])

    @torch.no_grad()
    def evaluate(self, model):
        model.eval()
        # The task is to predict the last word of the input.
        total, hit = 0, 0
        for batch in self.dataset:
            input_ids = batch["input_ids"].to(self.device).unsqueeze(0)
            label = input_ids[:, -1]
            outputs = model(input_ids)
            last_token_logits = outputs.logits[:, -2, :]
            pred = last_token_logits.argmax(dim=-1)
            total += label.size(0)
            hit += (pred == label).sum().item()
        acc = hit / total
        return acc

In [ ]:
from datasets import load_dataset

model_name = "facebook/opt-125m"
acc_tokenizer = GPT2Tokenizer.from_pretrained(model_name)
perp_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
acc_dataset = load_dataset("lambada", split="validation[:40]")
perp_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
acc_evaluator = AccuracyEvaluator(acc_dataset, acc_tokenizer, device)
perp_evaluator = PerplexityEvaluator(perp_dataset, perp_tokenizer, device, n_samples=15)

## FP16 Model Accuracy

Let's first check the performance of the original FP16 model.

In [ ]:
model_fp16 = OPTForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)
res_fp16 = perp_evaluator.evaluate(model_fp16)
print(f"Original model (fp16) result: {res_fp16}")

We then quantize the model to W8A8 and check the performance.

## Naive W8A8 Quantized Model Accuracy

In [ ]:
model_fp16 = OPTForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)

n_bits = 4
q_group_size = 128 # 0 means no group
q_protect = True # False means no protection
q_protection_ratio = 0.01 # 0.01 means 1% of the weights are protected.
q_protection_scale = 0.0 # 0.0 mixed-precision. >1.0 means scale up/down.
q_name = f"W{n_bits}A{n_bits}"
q_model = quantize_opt(
    model_fp16,
    n_bits=n_bits,
    q_group_size=q_group_size,
    q_protect=q_protect,
    q_protection_ratio=q_protection_ratio,
    q_protection_scale=q_protection_scale,
)
q_res = perp_evaluator.evaluate(q_model)
print(f"Naive {q_name} quantized model result: {q_res}")

In [ ]:
print(q_model)

We can see there is a significant accuracy drop. This is consistent with LLM.int8()'s finding: when the model size increases larger than 6.7B, systematic outliers will emerge in activations, which makes fully INT8 quantization impossible.

## SmoothQuant W8A8 Quantized Model Accuracy

Let's smooth the model, quantize it, and check the performance! In `../act_scales`, we provide the activation scales for OPT and BLOOM models. You can also use this notebook to test quantizing those models.

In [ ]:
model = OPTForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)
scales_path = "smoothquant/act_scales/opt-125m.pt"
act_scales = torch.load(scales_path)
smooth_lm(model, act_scales, 0.5)
q_model_smooth = quantize_opt(
    model,
    n_bits=n_bits,
    q_group_size=q_group_size,
    q_protect=q_protect,
    q_protection_ratio=q_protection_ratio,
    q_protection_scale=q_protection_scale,
)
q_res_smooth = perp_evaluator.evaluate(q_model_smooth)
print(f"Smoothed {q_name} quantized model result: {q_res_smooth}")

We can see the smoothed model has the same accuracy as the FP16 model. This is because SmoothQuant smooths the outliers in activations and moves the quantization difficulty from activations to weights.